# Coinbase Pro API Test

The sandbox version of the coinbase Pro is at:

   https://public.sandbox.pro.coinbase.com
    
Cbpro Python library repository:

   https://github.com/danpaquin/coinbasepro-python
    
API url (sandbox):

   https://api-public.sandbox.pro.coinbase.com
   
Websocket url (sandbox):
   
   [wss://ws-feed-public.sandbox.pro.coinbase.com](wss://ws-feed-public.sandbox.pro.coinbase.com)

In [6]:
%config Completer.use_jedi = False       #without this line, autocomplete seemed to not be working, so I am putting it here. source: https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working

In [7]:
import cbpro
from auth_credentials import (api_secret,
                              api_key, 
                              api_pass)

## WebsocketClient Class

Here we inherit from the WebsocketClient class and overload methods `on_open()`, `on_message()`, and `on_close()`

In [8]:
class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url           = 'wss://ws-feed-public.sandbox.pro.coinbase.com'
        self.message_count = 0
    
    def on_message(self,msg):
        self.message_count += 1
        msg_type = msg.get('type',None)
        if msg_type == 'ticker':
            time_val   = msg.get('time',('-'*27))
            price_val  = msg.get('price',None)
            if price_val is not None:
                price_val = float(price_val)
            product_id = msg.get('product_id',None)
            
            print(f"{time_val:30} \
                {price_val:.3f} \
                {product_id}\tchannel type:{msg_type}")
    
    def on_close(self):
        print(f"<---Websocket connection closed--->\n\tTotal messages: {self.message_count}")

In [9]:
stream = TextWebsocketClient(products=['BTC-USD'],channels=['ticker'])
stream.start()

2021-02-08T08:21:05.040511Z                    39201.920                 BTC-USD	channel type:ticker
2021-02-08T08:21:06.394668Z                    39201.910                 BTC-USD	channel type:ticker
2021-02-08T08:21:07.774882Z                    39201.920                 BTC-USD	channel type:ticker
2021-02-08T08:21:08.353162Z                    39201.920                 BTC-USD	channel type:ticker
2021-02-08T08:21:09.161940Z                    39201.920                 BTC-USD	channel type:ticker
2021-02-08T08:21:10.264248Z                    39201.920                 BTC-USD	channel type:ticker
2021-02-08T08:21:10.290122Z                    39201.920                 BTC-USD	channel type:ticker
2021-02-08T08:21:11.082410Z                    39201.920                 BTC-USD	channel type:ticker
2021-02-08T08:21:11.396322Z                    39201.920                 BTC-USD	channel type:ticker
2021-02-08T08:21:11.627855Z                    39201.920                 BTC-USD	channel ty

In [10]:
stream.close()

2021-02-08T08:21:17.360564Z                    39201.920                 BTC-USD	channel type:ticker
<---Websocket connection closed--->
	Total messages: 20


## AuthenticatedClient Class


An authenticated client allows us to access private account information and can be done with the class `AuthenticatedClient` to handle this type of connection.

In [20]:
url='https://api-public.sandbox.pro.coinbase.com'

client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url=url
)

## Placing a Market Order

Here we see how to place market orders through cbpro with an authenticated client object.

In [ ]:
client.place_market_order(product_id='BTC-USD',side='buy',size=1)

## Check Account History

First we need to know the id of the account we want to check. Therefore, we use `client.get_accounts()` to obtain the id of the target account and we then check its history with `client.get_account_history(<account id>)`.

In [32]:
accounts = client.get_accounts()

In [33]:
for acc in accounts:
    currency = acc.get('currency')
    if currency=='BTC':
        acc_id = acc.get('id')

In [34]:
acc_history = client.get_account_history(acc_id)

In [ ]:
import json

for hist in acc_history:
    print(json.dumps(hist,indent=1))